In [1]:
# your other imports here ...
import sys, os
import pandas as pd
import matplotlib.pyplot as plt

# TODO: replace with your path/to/ninarow
ninarowdir = os.path.dirname(os.getcwd())
modelfitdir = ninarowdir + "/model_fitting/"
# os.listdir(modelfitdir)

# sets the import path to the model-fitting directory
sys.path.insert(0, modelfitdir)
from parsers import *
# from new_fit import *
from utils import *
from utils import show
from tqdm import tqdm
import random

# WARNING: %load_ext autoreload and %autoreload 2 may interfere with 
# the Multi-threading processes!
%load_ext autoreload
%autoreload 2

In [3]:
df = pd.read_csv("../data/data.csv")
df["response_time"] = 1
df = df[:100]
df.head()
make_splits(df, 5, "../data");

Saving split1 to ../data/1.csv
Saving split2 to ../data/2.csv
Saving split3 to ../data/3.csv
Saving split4 to ../data/4.csv
Saving split5 to ../data/5.csv


In [4]:
data_path = "../data"
output_path = "../data/out"
n_splits = 5
fold_number = 1
threads = 1
random_sample = False
verbose = True

print(f"Building output directory at {output_path}")
os.makedirs(output_path, exist_ok = True)

# first, we have to check to see if all the splits are there ...
assert np.all([f"{i + 1}.csv" in os.listdir(data_path) for i in range(n_splits)])
print("Detected splits in this directory. Loading splits ...")

# then we read them in
splits = [pd.read_csv(f"{data_path}/{i + 1}.csv") for i in range(n_splits)]

# we convert every row of our CSV to a "CSVMove object" using df_to_CSVMove - we do so for all the splits
# CSVMove is a class that is defined in the parsers.py file 
fold_data = [[csvmove for csvmove in df_to_CSVMove(split, warn = True)] for split in splits]

Building output directory at ../data/out
Detected splits in this directory. Loading splits ...


In [8]:
import model
from time import time
# from utils import *

random.seed(10)
model.initialize_thread_pool(1, manual_seed = 10)
model_fitter =  model.Fitter(model.DefaultModel(), threads = 1, verbose = True)

tick = time()
# q = model.cross_validate(model.DefaultModel(), splits, leave_out_idx = 4, threads = 1)
model_fitter.fit(pd.concat([splits[0], splits[2], splits[3], splits[4]]))
print(time() - tick)

Setting manual seed 10 for single-thread
Thread 0: Base Seed 10, Seed: 10, Random Number: 601088376405717203

[Preprocessing] Initial log-likelihood estimation


 10%|█         | 1/10 [00:03<00:29,  3.25s/it]

	Time taken: 3.2489421367645264 since Start, 3.2448980808258057 since Loop


 20%|██        | 2/10 [00:06<00:25,  3.17s/it]

	Time taken: 6.372497081756592 since Start, 3.121752977371216 since Loop


...

	Time taken: 32.61838102340698 since Start, 3.8646888732910156 since Loop


100%|██████████| 10/10 [00:35<00:00,  3.56s/it]

	Time taken: 35.5662362575531 since Start, 2.9458460807800293 since Loop


	Time taken: 47.662986040115356 since Start, 11.759611129760742 since Loop
	[0] NLL: 145.811 Params: [2.001, 0.02, 0.2, 0.05, 1.2, 0.801, 1.001, 0.4, 3.501, 5.0]
Beginning optimization of a STOCHASTIC objective function

 Iteration    f-count      E[f(x)]        SD[f(x)]           MeshScale          Method              Actions
     0           1         145.811             nan               1                                  


KeyboardInterrupt: 

In [9]:
import model_fit
from time import time
from parsers import *

class DotDict(dict):
    """A dictionary that supports dot notation access."""
    def __getattr__(self, name):
        if name in self:
            return self[name]
        raise AttributeError(f"'{self.__class__.__name__}' object has no attribute '{name}'")
    
    def __setattr__(self, name, value):
        self[name] = value


args = DotDict({"random_sample": random_sample, 
    "verbose": verbose, 
    "threads": threads})

random.seed(10)
model_fit.initialize_thread_pool(1, manual_seed = 10)
model_fitter =  model_fit.ModelFitter(model_fit.DefaultModel(), args)

tick = time()
model_fitter.fit_model(fold_data[0] + fold_data[2] + fold_data[3] + fold_data[4])
print(time() - tick)

Setting manual seed 10 for single-thread
Thread 0: Base Seed 10, Seed: 10, Random Number: 601088376405717203

Beginning model fit pre-processing: log-likelihood estimation


 10%|█         | 1/10 [00:03<00:31,  3.48s/it]

	Time taken: 3.481494903564453 since Start, 3.4753458499908447 since Loop


 20%|██        | 2/10 [00:06<00:27,  3.39s/it]

	Time taken: 6.815514087677002 since Start, 3.3292007446289062 since Loop


...

	Time taken: 34.596230030059814 since Start, 4.099689960479736 since Loop


100%|██████████| 10/10 [00:37<00:00,  3.78s/it]


	Time taken: 37.77687907218933 since Start, 3.172827959060669 since Loop


100%|██████████| 80/80 [00:00<00:00, 6956.74it/s]


KeyboardInterrupt: 